# LDA
by Esteban Ariza Acosta

In [18]:
%pip install pyldavis
%pip install pyldavis.gensim
%pip install spacy

Note: you may need to restart the kernel to use updated packages.Requirement already satisfied: pyldavis in c:\users\ariza\anaconda3\lib\site-packages (3.3.1)




[notice] A new release of pip available: 22.2.2 -> 23.0
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement pyldavis.gensim (from versions: none)
ERROR: No matching distribution found for pyldavis.gensim

[notice] A new release of pip available: 22.2.2 -> 23.0
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.2.2 -> 23.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import gensim
import gensim.corpora as corpora
from gensim.corpora import Dictionary
from gensim.models.coherencemodel import CoherenceModel
from gensim.models.ldamodel import LdaModel

from pprint import pprint

import spacy

import pickle
import re
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis

import matplotlib.pyplot as plt
import pandas as pd

from nltk.corpus import stopwords

import string

In [28]:
df = pd.read_csv("../data/exploratory_analysis/tripadvisor_hotels_clean.csv")
reviews = df["REVIEW_TEXT"].values.tolist()
hotels = df['HOTEL_NAME'].values.tolist()

#Eliminate puntiation marks
reviews = [r.translate(str.maketrans('','',string.punctuation)) for r in reviews]

#Lower case
reviews = [r.lower() for r in reviews]

#Split words
reviews = [r.split(' ') for r in reviews]

#Blank spaces
reviews = [list(filter(lambda r: not (not r), review)) for review in reviews]

In [29]:
# Eliminate common words
sw = list(set(stopwords.words('english')))
reviews = [list(filter(lambda r: r not in sw, review)) for review in reviews]

In [58]:
id2word = Dictionary(reviews)
# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in reviews]
print(corpus[:2])

[[(0, 1), (1, 1), (2, 2), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 1), (29, 1), (30, 2), (31, 1), (32, 2), (33, 2), (34, 1), (35, 1), (36, 1), (37, 1), (38, 1)], [(31, 1), (32, 1), (39, 1), (40, 1), (41, 1), (42, 1), (43, 1), (44, 1), (45, 1), (46, 1), (47, 1), (48, 1), (49, 1), (50, 1), (51, 1), (52, 1), (53, 1), (54, 1), (55, 1), (56, 1), (57, 1), (58, 1), (59, 1), (60, 1), (61, 1), (62, 1), (63, 1), (64, 1), (65, 1), (66, 1), (67, 1)]]


In [31]:
[[(id2word[i], freq) for i, freq in doc] for doc in corpus[:1]]

[[('5', 1),
  ('area', 1),
  ('beautiful', 2),
  ('book', 1),
  ('breakfast', 1),
  ('class', 1),
  ('comes', 1),
  ('doubt', 1),
  ('ever', 1),
  ('experience', 1),
  ('gardens', 1),
  ('gives', 1),
  ('helpful', 1),
  ('high', 1),
  ('highly', 1),
  ('hotels', 1),
  ('immaculate', 1),
  ('loved', 1),
  ('lovely', 1),
  ('many', 1),
  ('nights', 1),
  ('none', 1),
  ('one', 1),
  ('place', 1),
  ('really', 1),
  ('recommend', 1),
  ('restaurant', 1),
  ('rooms', 1),
  ('second', 1),
  ('something', 1),
  ('special', 2),
  ('staff', 1),
  ('stay', 2),
  ('stayed', 2),
  ('surround', 1),
  ('unique', 1),
  ('visit', 1),
  ('without', 1),
  ('would', 1)]]

In [33]:
CLUSTERS = 6
# Build LDA model
lda_model = LdaModel(corpus=corpus,
                   id2word=id2word,
                   num_topics=CLUSTERS,
                   random_state=0,
                   chunksize=100,
                   alpha='auto',
                   per_word_topics=True)

# pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

In [34]:
class_data = []
for i in range(len(doc_lda)):
    act_class_data = [hotels[i]]
    topics = doc_lda[i][0]
    for topic in topics:
        act_class_data.append(topic[1])
    class_data.append(act_class_data)

cdf_columns =[str(i+1) for i in range(CLUSTERS)]
cdf_columns.insert(0, "HOTEL")
cdf = pd.DataFrame(data=class_data, columns=cdf_columns)

In [63]:
cdf["CLUSTER"] = cdf["1"] + cdf["2"] + cdf["3"] + cdf["4"] + cdf["5"] + cdf["6"]
doc_lda[2][0]

[(0, 0.12048311),
 (1, 0.2094629),
 (2, 0.031570066),
 (4, 0.19278856),
 (5, 0.4373676)]

In [50]:
gcdf = cdf.groupby('HOTEL').mean()
gcdf.to_csv(f"lda_prob_{CLUSTERS}.csv")

In [56]:
# Import hotel cluster
def cluster_map(row):
    values = [i for i in row[1:]]
    max_value = max(values)
    return values.index(max_value)+1


gcdf = pd.read_csv("lda/lda_prob_6.csv")

gcdf["CLUSTER"] = gcdf.apply(cluster_map, axis=1)

gcdf.sort_values(by="CLUSTER")

,HOTEL,1,2,3,4,5,6,CLUSTER
0,3HB Faro,0.201637,0.192898,0.045412,0.033769,0.139302,0.475833,6
221,Mamaison Hotel Le Regina Warsaw,0.178916,0.182875,0.059620,0.028977,0.123459,0.455662,6
220,Maison Albar Hotels - Le Monumental Palace,0.167725,0.179119,0.058224,0.029964,0.110177,0.490919,6
219,Maalot Roma - Small Luxury Hotels Of The World,0.178298,0.185120,0.051825,0.038756,0.177205,0.499778,6
218,MYRIAD by SANA,0.182311,0.197988,0.061070,0.035530,0.139505,0.445517,6
...,...,...,...,...,...,...,...,...
105,Grandior Hotel Prague,0.205038,0.183712,0.043876,0.033977,0.119716,0.450365,6
104,Grandhotel Pupp,0.186114,0.229052,0.070596,0.033357,0.105375,0.409530,6
103,"Grand-Hotel du Cap-Ferrat, A Four Seasons Hotel",0.154359,0.222501,0.059298,0.035983,0.110198,0.450105,6
121,Hotel Can Cera Adults only,0.170289,0.174703,0.064522,0.030396,0.120219,0.475100,6


In [26]:
# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=reviews, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Coherence Score:  0.3697462750071752


In [43]:
#Creating Topic Distance Visualization 
pyLDAvis.enable_notebook()
prepare_data = gensimvis.prepare(lda_model, corpus, id2word, mds='mmds') #R=60
# html = pyLDAvis.prepared_data_to_html(prepare_data)
pyLDAvis.display(prepare_data)

c:\Users\ariza\anaconda3\lib\site-packages\pyLDAvis\_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  default_term_info = default_term_info.sort_values(


In [ ]:
#write string to file
text_file = open(f"lda_{str(CLUSTERS)}.html", "w")
text_file.write(html)
text_file.close()

## Examples

In [36]:
sum_df = pd.read_csv("../data/review_summarizer/summarized_reviews_by_year_and_hotel-small.csv");
in_df = pd.read_csv("../data/exploratory_analysis/tripadvisor_hotels_clean.csv")

In [77]:
REVIEW_CONCATCHAR1 = "\n"

def fromDateToYear(value): #Clean CSV (yyyy-mm-dd)
    return value.split("-")[0]

def concatReviewsByYearAndHotel(df):
    df = df.copy()
    df["REVIEW_DATE"] = df["REVIEW_DATE"].map(fromDateToYear).astype(int)
    df['REVIEW_TEXT'] = df[['HOTEL_NAME','REVIEW_TEXT','REVIEW_DATE']].groupby(["HOTEL_NAME","REVIEW_DATE"])["REVIEW_TEXT"].transform(lambda x: REVIEW_CONCATCHAR1.join(x))
    return df[['HOTEL_NAME','REVIEW_DATE','REVIEW_TEXT']].drop_duplicates()

# Group by NAME and YEAR
iny_df = concatReviewsByYearAndHotel(in_df.dropna())

In [54]:
sum_df

,HOTEL_NAME,REVIEW_DATE,REVIEW_TEXT,REVIEW_SUMMARY
0,Corpo Santo Lisbon Historical Hotel,2022,we loved the place and loved it again this tim...,the staff at the Corpo Santo Historical Hotel ...
1,Kimpton De Witt Amsterdam,2019,the hotel is unique in style and comfortable. ...,"the hotel is located in the heart of the city,..."
2,Corinthia Budapest,2019,hotel moments Budapest was a great place to st...,the corinthia hotel in the heart of the city i...
3,Grandior Hotel Prague,2019,the room was excellent and the buffet breakfas...,the hotel is located in the heart of the old c...
4,Hilton Budapest,2019,the hotel is built into the rocks in the ruins...,the hotel is located in the heart of the castl...
...,...,...,...,...
2627,The Guesthouse Vienna,2015,The best location to be in old Vienna. You can...,this is one of the most beautiful cities in th...
2628,Andreus Resort,2021,"If you need the ""perfection"" during a Holiday,...","Andreus Resort is a luxury resort, with easy m..."
2629,Mystery Hotel Budapest,2018,The hotel is fabolous and has a very unique de...,the hotel is fabolous and has a very unique de...
2630,Olissippo Lapa Palace,2014,"Huge bedrooms, really comfortable, wonderful g...",Lisbon is a wonderful city to visit. no hesits...


In [98]:
sum_sample = sum_df.sample(1).iloc[0]

sum_in_sample = iny_df[(iny_df["REVIEW_DATE"]==sum_sample["REVIEW_DATE"]) & (iny_df["HOTEL_NAME"]==sum_sample["HOTEL_NAME"])].iloc[0]

print("IN:")
print(sum_in_sample["REVIEW_TEXT"])
print("OUT:")
print(sum_sample["REVIEW_SUMMARY"])

IN:
We spent a long weekend at the Fontenay for my birthday and consider it really first class.  Beautiful suite, very fair family offer for an additional connecting room. Service is great, top concierge and we loved the pool at the rooftop.   Only the service at the rooftop bar was not that good...
Stayed for two nights on a recent business trip to Hamburg  An all round first class experience, beginning at the check in and continuing throughout our stay. Rooms are very well equipped, and very comfortable. Buffet breakfast on ground floor is superb, dare I say, one of the best I have experienced. Bar area on 6th floor also great. Highly recommended for business or pleasure.
Really a top of the line hotel with a great location, staff that give you the feeling that they care and take time for you.  A short walk to the city.  The spa area is relaxing and a good intro to the day.    A great choice  
Wonderful weekend break - visited the 6th floor bar for a super chicken Caesar salad in sun